In [1]:
import numpy as np
import sys
sys.path.append('..')

from utils.data import fetch_data_with_label_per_step, label_to_int, balance_data
from utils.clustering import Clustering
from utils.encoding import make_good_unitary, encode_feature
from sklearn.metrics import adjusted_rand_score
import itertools
import argparse

logfile_path = ['../data/raw/1599166289/data.pickle', '../data/raw/1599153598/data.pickle', '../data/raw/test/data.pickle']

#coordinates and object kind for nut and bolt
data, label = fetch_data_with_label_per_step(logfile_path)

In [42]:
n = len(data)
n_training = 5
inds = np.random.choice(n, n_training)
data_training = np.zeros((1, 4))
label_training = []

for i, ind in enumerate(inds):
    selected_data = np.array(list(itertools.chain.from_iterable(data[ind])))
    selected_label = list(itertools.chain.from_iterable(label[ind]))
    data_training = np.vstack((data_training, selected_data))
    label_training += selected_label

data_training = data_training[1:,:]
label_training = np.array(label_training)


selected_data_balanced, selected_label_balanced  = balance_data(data_training, label_training)
selected_label_int = label_to_int(label_training_balanced)


u_coord = 0  # The average shift between the approximated coordinates and ground truth
sigma_coord = 0.006
u_kind = 0.1
sigma_kind = 0.05
u_anchor = 0.1
sigma_anchor = 0.05

coord = selected_data_balanced[:,0:3]
obj_kind = selected_data_balanced[:, 3]

noise = np.random.normal(u_coord, sigma_coord, coord.shape)
coord_noisy = coord + noise

ind_nut = np.where(obj_kind == 0)[0]
ind_bolt = np.where(obj_kind == 1)[0]

n_data = coord.shape[0]
one_hot_kind = np.zeros((n_data,2))
one_hot_kind[np.arange(obj_kind.size),obj_kind.astype(int)] = 1
noise_kind = np.random.normal(u_kind, sigma_kind, obj_kind.size)
kind_noisy = abs(one_hot_kind - np.column_stack((noise_kind, noise_kind)))

ind_table = np.concatenate((np.where(selected_label_balanced == 'Nut on table')[0], np.where(selected_label_balanced == 'Bolt on table')[0]))
ind_jig = np.delete(np.arange(n_data), ind_table)
one_hot_anchor = np.zeros((n_data,2))
one_hot_anchor[ind_table,0] = 1
one_hot_anchor[ind_jig,1] = 1
noise_anchor = np.random.normal(u_anchor, sigma_anchor, obj_kind.size)
anchor_noisy = abs(one_hot_anchor - np.column_stack((noise_anchor, noise_anchor)))

coord_concat_ext = np.concatenate((coord_noisy, kind_noisy, anchor_noisy), axis = 1)
